# Prototyping an ML Model on Tensorflow Datasets
## Prerequisites

In [1]:
import gc
import glob
import os
from typing import Iterable, Callable, Dict, Any, Tuple, Optional, List, Union

import numpy as np
import pandas as pd
import tensorflow as tf

from mmproteo.utils import log
from mmproteo.utils.formats.mz import FilteringProcessor
from mmproteo.utils.processing import ItemProcessor

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [3]:
logger = log.DummyLogger(verbose=False)

INFO: Printing to Stdout


## Configuration

In [4]:
pwd

'/tf/workspace/notebooks'

In [5]:
PROJECT = "PXD010000"
DUMP_PATH = os.path.join("..", "dumps", PROJECT)
TRAINING_COLUMNS_DUMP_PATH = os.path.join(DUMP_PATH, "training_columns")
FILES_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "*_mzmlid.parquet")
STATISTICS_FILE_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "statistics.parquet")
DATASET_DUMP_PATH = os.path.join(TRAINING_COLUMNS_DUMP_PATH, "tf_datasets")

In [6]:
MZMLID_FILE_PATHS = glob.glob(FILES_PATH)
len(MZMLID_FILE_PATHS)

40

In [7]:
MZMLID_FILE_PATHS[0]

'../dumps/PXD010000/training_columns/Biodiversity_B_fragilis_01_28Jul15_Arwen_14-12-03_mzmlid.parquet'

In [8]:
pd.read_parquet(MZMLID_FILE_PATHS[1]).head(2)

,peptide_sequence,mz_array,intensity_array,species,istrain
6,"[L, D, N, V, V, Y, R]","[100.03951, 100.07604, 100.08698, 101.0598, 101.07139, 101.107635, 102.05545, 107.04927, 110.07147, 112.050835, 112.07621, 112.07955, 112.08708, 112.11265, 113.07123, 114.05504, 114.10244, 115.0504, 115.08685, 116.07068, 116.97243, 117.10215, 117.8191, 119.049065, 120.08085, 121.08424, 126.054794, 126.06571, 127.0868, 127.095535, 128.07236, 128.08185, 129.06583, 129.1024, 129.1124, 130.0507, 130.08633, 130.09756, 130.10551, 133.06154, 133.09709, 136.07576, 137.07903, 138.06609, 138.0916, 139.08571, 139.69499, 140.0812, 140.14333, 141.06573, 141.1022, 143.08153, 143.11768, 145.06099, 145.09743, 147.11273, 152.07062, 153.06499, 155.08113, 155.1178, 156.10173, 157.09727, 157.10855, 157.13348, 157.14546, 158.08061, 158.09245, 158.13683, 159.07657, 159.09282, 159.11234, 165.1023, 166.06062, 169.08437, 169.09724, 169.13377, 171.07635, 171.11241, 171.14874, 173.0913, 173.12836, 175.11905, 176.12291, 177.10197, 180.06554, 181.09618, 181.13329, 183.11295, 184.09566, 184.11569, 185.05496, 185.12808, 185.16528, 186.1237, 187.07101, 187.10844, 187.12733, 187.1442, 191.11745, 193.09688, ...]","[1472.0198, 1778.061, 982.26117, 849.2956, 7433.908, 1517.598, 10654.481, 1285.867, 22276.096, 1036.7197, 1008.01794, 1077.3555, 17357.316, 1765.9006, 1840.2622, 884.15027, 1030.9141, 1466.3262, 15643.766, 12685.864, 884.1921, 1245.8772, 909.2175, 1600.0315, 22742.06, 1201.5543, 1346.1736, 790.16095, 7063.337, 811.5802, 1109.8296, 6045.0728, 4442.3276, 33509.23, 2367.9402, 1291.7451, 11317.632, 6297.992, 2274.9944, 1097.1204, 1409.3613, 63035.684, 3738.175, 901.6442, 1888.8175, 784.02856, 840.27997, 875.71985, 1409.5555, 2824.3079, 8497.284, 4310.6895, 6666.118, 1519.9, 967.4869, 12989.814, 5915.0684, 942.62933, 4556.132, 1359.317, 5664.3735, 1361.2703, 5972.5083, 20817.252, 1598.2277, 1266.6726, 26993.193, 1077.943, 1951.6704, 796.94104, 1074.771, 1259.3566, 1369.4341, 1098.0599, 14032.469, 3526.5652, 1029.5724, 5769.34, 4063.4348, 1824.1956, 7088.784, 141361.95, 6137.2266, 1245.3956, 1576.3531, 2777.617, 1268.7177, 65213.92, 894.09875, 5262.379, 4305.7144, 1807.8231, 3397.9797, 43168.617, 1053.397, 3030.1729, 3255.7651, 3606.3943, 1305.8389, 4084.5789, ...]",Alcaligenes_faecalis,Train
7,"[A, G, L, D, N, N, Y, V, K]","[100.03982, 100.07586, 101.071236, 101.107574, 102.05517, 107.04923, 110.0715, 111.05547, 112.0509, 112.08704, 113.071, 115.086555, 116.07075, 116.9723, 119.04964, 120.08085, 127.08648, 128.07094, 128.08191, 129.06592, 129.10237, 130.04881, 130.07718, 130.08636, 130.09673, 130.10614, 136.04158, 136.0756, 137.07896, 139.04985, 141.06577, 141.10187, 142.12172, 143.08133, 143.11786, 144.12074, 147.11266, 148.11627, 152.07063, 155.08133, 155.11841, 157.06078, 157.09642, 157.13316, 158.09187, 159.07617, 159.11238, 166.06183, 169.09738, 169.13321, 171.11256, 171.14896, 173.09091, 173.12814, 173.97696, 174.13239, 175.1189, 181.06166, 181.09688, 183.11264, 185.09157, 186.0881, 186.12384, 187.10779, 187.14383, 195.07558, 195.11305, 197.12822, 201.0985, 201.12306, 202.0831, 204.13431, 211.14339, 212.06625, 212.10194, 215.1022, 215.13902, 216.10397, 223.15463, 226.11867, 227.10811, 228.13367, 228.17004, 229.09404, 229.11703, 230.07692, 230.11682, 231.07915, 233.09244, 235.14424, 242.11343, 242.14897, 244.16553, 246.15945, 246.1812, 247.10516, 247.18224, 250.11873, 254.14992, 255.1447, ...]","[732.6617, 1411.32, 16631.832, 914.66113, 1730.3816, 1458.0236, 11018.118, 814.45605, 824.7896, 4943.581, 1140.477, 3491.9822, 809.35693, 1004.9697, 907.5157, 4783.916, 1493.2881, 1060.6812, 2808.3154, 18674.494, 37496.016, 1481.8646, 1417.5668, 31441.13, 1304.8645, 798.3385, 861.07764, 15419.474, 735.78265, 858.0814, 3415.7173, 1404.7439, 869.0062, 1809.3257, 44420.22, 3551.6487, 40098.707, 1224.2285, 1093.3992, 3180.0164, 834.1555, 840.0687, 1675.8429, 3058.9111, 3239.9705, 3116.7256, 899.7371, 752.41925, 5850.0176, 2989.287, 12923.561, 752.36975, 1145.8423, 3916.3987, 751

In [9]:
SEQ = 'peptide_sequence'
MZ = FilteringProcessor.default_mz_array_column_name
INT = FilteringProcessor.default_intensity_array_column_name

## Calculating Statistics over all MZMLID Files

In [10]:
file_path_count = len(MZMLID_FILE_PATHS)

def get_mzmlid_file_stats(item: Tuple[int, str]) -> Dict[str, Any]:
    idx, path = item
    info_text = f"Processing item {idx + 1}/{file_path_count} '{path}'"
    if idx % 10 == 0:
        logger.info(info_text)
    else:
        logger.debug(info_text)
    df = pd.read_parquet(path)
    max_sequence_length = df[SEQ].str.len().max()
    max_array_length = df[INT].str.len().max()
    alphabet = set.union(*df[SEQ].apply(set))
    item_count = len(df)
    del df
    gc.collect()
    
    return {
        "file_path": path,
        "max_sequence_length": max_sequence_length,
        "max_array_length": max_array_length,
        "alphabet": alphabet,
        "item_count": item_count
    }

if os.path.exists(STATISTICS_FILE_PATH):
    file_stats = pd.read_parquet(STATISTICS_FILE_PATH)
    file_stats.alphabet = file_stats.alphabet.apply(set)
    print(f"loaded previous statistics file '{STATISTICS_FILE_PATH}'")
else:
    file_stats = pd.DataFrame(
        ItemProcessor(
            items=enumerate(MZMLID_FILE_PATHS),
            item_processor=get_mzmlid_file_stats,
            action_name="analyse",
            subject_name="mzmlid file",
            thread_count=0,
            logger=logger
        ).process()
    )
    
    file_stats_writable = file_stats.copy()
    file_stats_writable.alphabet = file_stats_writable.alphabet.apply(list) # cannot store sets
    file_stats_writable.to_parquet(STATISTICS_FILE_PATH)

loaded previous statistics file '../dumps/PXD010000/training_columns/statistics.parquet'


In [11]:
file_stats.head(2)

,file_path,max_sequence_length,max_array_length,alphabet,item_count
0,../dumps/PXD010000/training_columns/Biodiversity_B_fragilis_01_28Jul15_Arwen_14-12-03_mzmlid.parquet,50,1845,"{G, I, T, R, M(Oxidation), L, K, N, P, W, Y, M, F, E, V, S, A, D, C, Q, H}",26943
1,../dumps/PXD010000/training_columns/Biodiversity_A_faecalis_LB_aerobic_03_26Feb16_Arwen_16-01-01_mzmlid.parquet,49,1082,"{G, I, T, R, M(Oxidation), L, K, N, P, W, Y, M, F, E, V, S, A, D, C, Q, H}",16723


In [12]:
MAX_SEQUENCE_LENGTH = file_stats.max_sequence_length.max()
print(f"MAX_SEQUENCE_LENGTH = {MAX_SEQUENCE_LENGTH}")

MAX_ARRAY_LENGTH = file_stats.max_array_length.max()
print(f"MAX_ARRAY_LENGTH = {MAX_ARRAY_LENGTH}")

TOTAL_ITEM_COUNT = file_stats.item_count.sum()
print(f"TOTAL_ITEM_COUNT = {TOTAL_ITEM_COUNT}")

ALPHABET = set.union(*file_stats.alphabet)
print(f"ALPHABET = {', '.join(sorted(ALPHABET))}")

MAX_SEQUENCE_LENGTH = 50
MAX_ARRAY_LENGTH = 1845
TOTAL_ITEM_COUNT = 820586
ALPHABET = A, C, D, E, F, G, H, I, K, L, M, M(Oxidation), N, P, Q, R, S, T, V, W, Y


## Data Normalization, Padding, and Conversion to Tensorflow Datasets

In [13]:
def l2_normalize(values: np.ndarray) -> np.ndarray:
    return tf.keras.utils.normalize(x=values, order=2)

def base_peak_normalize(values: np.ndarray) -> np.ndarray:
    return values / values.max(initial=0)

# by Tom, probably
# don't know, what it's based on
def ion_current_normalize(intensities: np.ndarray) -> np.ndarray:
    total_sum = np.sum(intensities**2)
    normalized = intensities/total_sum
    return normalized

NORMALIZATION = {
    INT: base_peak_normalize
}

In [17]:
PADDING_CHARACTERS = {
    SEQ: '_',
    MZ: 0.0,
    INT: 0.0,
}

In [18]:
ALPHABET.add(PADDING_CHARACTERS[SEQ])

In [19]:
char_to_idx = {char: idx for idx, char in enumerate(sorted(ALPHABET))}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}
INDEX_ALPHABET = idx_to_char.keys()
char_to_idx

{'A': 0,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'K': 8,
 'L': 9,
 'M': 10,
 'M(Oxidation)': 11,
 'N': 12,
 'P': 13,
 'Q': 14,
 'R': 15,
 'S': 16,
 'T': 17,
 'V': 18,
 'W': 19,
 'Y': 20,
 '_': 21}

In [20]:
ARRAY_COLS = [MZ, INT]

In [21]:
# TODO: Transform to class

PADDING_LENGTHS = {
    MZ: MAX_ARRAY_LENGTH,
    INT: MAX_ARRAY_LENGTH,
    SEQ: MAX_SEQUENCE_LENGTH
}

class Parquet2DatasetFileProcessor:

    def __init__(self,
                 training_data_columns: List[str],
                 target_data_columns: List[str],
                 padding_lengths: Dict[str, int],
                 padding_characters: Dict[str, Union[str, int, float]],
                 column_normalizations: Dict[str, Callable[[Any], Any]],
                 char_to_idx_mappers: Optional[Dict[str, Dict[str, int]]] = None,
                 char_to_idx_mapping_functions: Optional[Dict[str, Callable[[str], int]]] = None,
                 item_count: int = 0,
                 dataset_dump_path: str = DATASET_DUMP_PATH,
                 training_columns_dump_path: str = TRAINING_COLUMNS_DUMP_PATH,
                 logger: log.Logger = logger
                 ):
        self.training_data_columns = training_data_columns
        self.target_data_columns = target_data_columns
        self.padding_lengths = padding_lengths
        self.padding_characters = padding_characters
        self.column_normalizations = column_normalizations
        self.char_to_idx_mapping_functions = char_to_idx_mapping_functions
        if self.char_to_idx_mapping_functions is None:
            assert char_to_idx_mappers is not None, \
                "either char_to_idx_mappers or char_to_idx_mapping_functions must be given"
            self.char_to_idx_mapping_functions = {
                column: mapping.get for column, mapping in char_to_idx_mappers.items()
            }
        self.item_count = item_count
        self.dataset_dump_path = dataset_dump_path
        self.training_columns_dump_path = training_columns_dump_path
        self.char_idx_dtype = np.int8
        self.logger = logger

    def normalize_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        if self.column_normalizations is None:
            return df
        df = df.copy()
        for column, normalize_func in self.column_normalizations.items():
            df[column] = df[column].apply(normalize_func)
        return df

    def pad_array_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        if len(df) == 0:
            return df

        df = df.copy()
        for column, padding_length in self.padding_lengths.items():
            item_dtype = df[column].iloc[0].dtype

            df[column] = list(tf.keras.preprocessing.sequence.pad_sequences(
                sequences=df[column],
                maxlen=padding_length,
                padding='post',
                value=self.padding_characters[column],
                dtype=item_dtype
            ))
        return df

    @staticmethod
    def _sequence_to_indices(sequence: Iterable[str],
                             char_to_idx_mapping_func: Callable[[str], int],
                             dtype: type) -> np.ndarray:
        return np.array([char_to_idx_mapping_func(char) for char in sequence],
                        dtype=dtype)

    def sequence_column_to_indices(self, df: pd.DataFrame) -> pd.DataFrame:
        if len(self.char_to_idx_mapping_functions) == 0:
            return df
        df = df.copy()
        for column, mapping_function in self.char_to_idx_mapping_functions.items():
            df[column] = df[column].apply(lambda seq: self._sequence_to_indices(seq,
                                                                                mapping_function,
                                                                                self.char_idx_dtype))
        return df

    @staticmethod
    def stack_numpy_arrays_in_dataframe(df: pd.DataFrame) -> pd.DataFrame:
        return df.apply(lambda item: [np.stack(item)])

    def preprocess_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self.normalize_columns(df)
        df = self.pad_array_columns(df)
        df = self.sequence_column_to_indices(df)
        df = self.stack_numpy_arrays_in_dataframe(df)
        return df

    def stacked_df_to_dataset(self, stacked_df: pd.DataFrame) -> tf.data.Dataset:
        assert len(stacked_df) == 1, "all column values should be stacked at this point"
        training_data = tuple(stacked_df[self.training_data_columns].iloc[0])
        target_data = tuple(stacked_df[self.target_data_columns].iloc[0])
        tf_dataset = tf.data.Dataset.from_tensor_slices((training_data, target_data))
        return tf_dataset

    def convert_df_file_to_dataset_file(self,
                                        df_input_file_path: str,
                                        tf_dataset_output_file_path: str):
        df = pd.read_parquet(df_input_file_path)
        df = self.preprocess_dataframe(df)
        tf_dataset = self.stacked_df_to_dataset(df)
        self.logger.debug(tf_dataset.element_spec)

        tf.data.experimental.save(dataset=tf_dataset,
                                  path=tf_dataset_output_file_path,
                                  compression='GZIP')

    def __call__(self, item: Tuple[int, str]) -> Optional[str]:
        idx, path = item
        tf_dataset_path = os.path.join(
            self.dataset_dump_path,
            path[len(self.training_columns_dump_path)+len(os.path.sep):])

        info_text = f"Processing item {idx + 1}/{self.item_count}: '{path}'"
        if idx % 10 == 0:
            self.logger.info(info_text)
        else:
            self.logger.debug(info_text)

        if os.path.exists(tf_dataset_path):
            self.logger.debug(f"Skipped '{path}' because '{tf_dataset_path}' already exists")
            return None

        self.convert_df_file_to_dataset_file(df_input_file_path=path,
                                             tf_dataset_output_file_path=tf_dataset_path)
        gc.collect()

        return tf_dataset_path

    def process(self, parquet_file_paths: Iterable[str], **kwargs) -> List[str]:
        item_processor = ItemProcessor(
            items=enumerate(parquet_file_paths),
            item_processor=self.__call__,
            action_name="parquet2tf_dataset-process",
            subject_name="mzmlid parquet file",
            logger=self.logger,
            **kwargs
        )
        results = list(item_processor.process())
        return results

In [22]:
# replace with function call to process #TODO
list(ItemProcessor(
    items=enumerate(MZMLID_FILE_PATHS),
    item_processor=parquet_file_to_dataset_file_converter,
    action_name="parquet2tf_dataset-process",
    subject_name="mzmlid parquet file",
    thread_count=2,
    logger=logger
).process())[:3]

INFO: No mzmlid parquet files were parquet2tf_dataset-processed
INFO: Encountered 0 exceptions during processing


[]

## Loading Tensorflow Datasets

In [23]:
dataset_file_paths = glob.glob(os.path.join(DATASET_DUMP_PATH, '*'))
print(len(dataset_file_paths))
dataset_file_paths[:3]

40


['../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_B_cereus_PN_L_CL_2_09Oct16_Pippin_16-05-06_mzmlid.parquet',
 '../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_A_tumefaciens_R2A_aerobic_2_23Nov16_Pippin_16-09-11_mzmlid.parquet',
 '../dumps/PXD010000/training_columns/tf_datasets/Biodiversity_B_fragilis_Carb_01_28Oct15_Arwen_15-07-13_mzmlid.parquet']

In [24]:
element_spec = ((tf.TensorSpec(shape=(MAX_ARRAY_LENGTH,), dtype=tf.float32), 
  tf.TensorSpec(shape=(MAX_ARRAY_LENGTH,), dtype=tf.float32)),
(tf.TensorSpec(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int8)))
element_spec

((TensorSpec(shape=(1845,), dtype=tf.float32, name=None),
  TensorSpec(shape=(1845,), dtype=tf.float32, name=None)),
 TensorSpec(shape=(50,), dtype=tf.int8, name=None))

In [25]:
datasets = [tf.data.experimental.load(path=path, element_spec=element_spec, compression='GZIP') 
            for path in dataset_file_paths]

In [26]:
datasets[:3]

[<_LoadDataset shapes: (((1845,), (1845,)), (50,)), types: ((tf.float32, tf.float32), tf.int8)>,
 <_LoadDataset shapes: (((1845,), (1845,)), (50,)), types: ((tf.float32, tf.float32), tf.int8)>,
 <_LoadDataset shapes: (((1845,), (1845,)), (50,)), types: ((tf.float32, tf.float32), tf.int8)>]

## Concatenating Tensorflow Datasets

In [27]:
BATCH_SIZE = 256

In [28]:
dataset = datasets[0]
for ds in datasets[1:]:
    dataset = dataset.concatenate(ds)

In [29]:
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: (((None, 1845), (None, 1845)), (None, 50)), types: ((tf.float32, tf.float32), tf.int8)>

In [30]:
assert False

AssertionError: 

## Building the Tensorflow Model

In [ ]:
input_layers = {col: tf.keras.layers.Input(shape=(MAX_ARRAY_LENGTH,)) for col in ARRAY_COLS}
input_layers

In [ ]:
x = input_layers[MZ] + input_layers[INT]

x = tf.keras.layers.Flatten()(x)

for _ in range(1):
    x = tf.keras.layers.Dense(16*MAX_SEQUENCE_LENGTH*len(ALPHABET))(x)
    x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(MAX_SEQUENCE_LENGTH*len(ALPHABET))(x)

x = tf.reshape(x,(-1, MAX_SEQUENCE_LENGTH, len(ALPHABET)))

x = tf.keras.activations.softmax(x)

model = tf.keras.Model([input_layers[MZ],input_layers[INT]],x)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy())
model.summary()

## Training the Tensorflow Model

In [ ]:
def split_dataset(dataset, fraction):
    split_value = int(len(dataset) * fraction)
    a = dataset.take(split_value)
    b = dataset.skip(split_value)
    return a, b

In [ ]:
dataset = dataset.shuffle(buffer_size=int(10000 / BATCH_SIZE))

In [ ]:
model.fit(dataset, epochs=10)

## Evaluating the Tensorflow Model